In [2]:
# Import the required packages
using JuMP
using CSV
using DataFrames
using PowerModels
using Ipopt
using Juniper

In [3]:
#Read in data
baseMVA = 1;

bus_data = CSV.read("bus_data.csv", DataFrame) |> Matrix;
(num_busses, var) = size(bus_data);
VMax = bus_data[:, 4];
VMin = bus_data[:, 7]; 
load_data = CSV.read("load_data.csv", DataFrame) |> Matrix;
(num_loads, var) = size(load_data);
branch_data = CSV.read("branch_data.csv", DataFrame) |> Matrix;
(num_branches, var) = size(branch_data)
G = CSV.read("G_mat.csv", DataFrame) |> Matrix;
B = CSV.read("B_mat.csv", DataFrame)|> Matrix;

#Expand the Pd data such that non load busses are given power demands of 0
Pd = load_data[:,5]*baseMVA
load_busses = load_data[:,2];
bus_idx = bus_data[:,2]; 
Pd_mod = zeros(1, num_busses);
for i = 1:num_busses
    idx = findall(isequal(bus_idx[i]), load_busses)
    if !isempty(idx)
        Pd_mod[i] = Pd[idx[1]]
    else
        Pd_mod[i] = 0;
    end
end
Pd = Pd_mod

Qd = load_data[:,4]*baseMVA
Qd_mod = zeros(1, num_busses);
for i = 1:num_busses
    idx = findall(isequal(bus_idx[i]), load_busses)
    if !isempty(idx)
        Qd_mod[i] = Qd[idx[1]]
    else
        Qd_mod[i] = 0;
    end
end
Qd = Qd_mod

1×4 Matrix{Float64}:
 0.0  0.2  0.2  0.2

In [ ]:
optimizer = Juniper.Optimizer
nl_solver = optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0)

model = Model(optimizer_with_attributes(optimizer, "nl_solver"=>nl_solver))

@variable(model, V[1:num_busses] >= 0)
@variable(model, theta[1:num_busses])
@variable(model, PG[1:num_busses] >= 0)
@variable(model, QG[1:num_busses] >= 0)
@variable(model, y[1:num_busses] >= 0)
@variable(model, x[1:num_busses], Bin)
@variable(model, Ploss[1:num_busses] >= 0)
@variable(model, Qloss[1:num_busses] >= 0)
@variable(model, I[1:num_busses, 1:num_busses] >= 0)
@variable(model, VP >= 0)

M = 10000;

#In bus data, a 3 indicates the slack bus, the connection to the grid
slack_idx = findall(isequal(3), bus_data[:,3])
non_slack_num = filter(x->x!=slack_idx, 1:num_busses)

@constraint(model, [i in 1:num_busses], y[i] <= M*x[i])
@NLconstraint(model, [i in 1:num_busses], PG[i] - Pd[i] + Ploss[i] == sum((V[i]*V[j])*(G[i,j]*cos(theta[i]-theta[j])+B[i,j]*sin(theta[i]-theta[j])) for j in 1:num_busses))
@constraint(model, [i in 1:num_busses], Ploss[i] <= Pd[i])
@NLconstraint(model, [i in 1:num_busses], QG[i] - Qd[i] == sum((V[i]*V[j])*(G[i,j]*sin(theta[i]-theta[j])-B[i,j]*cos(theta[i]-theta[j])) for j in 1:num_busses))
#This case has no flow limits
#@NLconstraint(model, [i in 1:num_busses, j in 1:num_busses], I[i,j] == sqrt(G[i,j]^2+B[i,j]^2)*sqrt(V[i]^2+V[j]^2-2*V[i]*V[j]*cos(theta[j]-theta[i])))
#When using capacity constraint sqrt(Pg^2+Qg^2) <= y, becomes infeasible
@constraint(model, [i in 1:num_busses], (PG[i]^2+QG[i]^2)^(1/2) <= y[i])
#This case has no flow limits
#@constraint(model, [i in 1:num_busses, j in 1:num_busses] I[i,j] <= Imax[i,j])
@constraint(model, [i in non_slack_num], VMin[i] <= V[i] <= VMax[i])
#@constraint(model, V[1] == 0)
#@constraint(model, theta[1] == 0)
@constraint(model, VP == sum(V[i]*Pd[i] for i in 1:num_busses))
@constraint(model, [i in 1:num_busses],  -pi/2 <= theta[i] <= pi/2 )

#Varying these parameters changes output behavior
#When gamma < 1, tends to just shed all load, bad!
#For gamma = 1, and slightly above sheds some load, for gamma well above 1, like 2, 
#Serves all load. Interesting! Hyper parameter tuning 
c_fix = 0;
c_var = 1;
beta = .5;
gamma = 1.5;

@objective(model, Min, sum(c_fix*x[i]+c_var*y[i] for i in 1:num_busses) +
                        beta*VP +
                        gamma*sum(Ploss[i] for i in 1:num_busses))

optimize!(model)


nl_solver         : MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("print_level") => 0])
feasibility_pump  : false
log_levels        : [:Options, :Table, :Info]

#Variables: 49
#IntBinVar: 4
Obj Sense: Min


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Start values are not feasible.
Status of relaxation: LOCALLY_SOLVED
Time for relaxation: 4.367000102996826
Relaxation Obj: 2.344447429400969

 ONodes   CLevel          Incumbent                   BestBound            Gap    Time   Restarts  GainGap  
    2       2                 -                   

In [8]:
JuMP.value.(QG)

4-element Vector{Float64}:
 8.915358123263802e-9
 0.20114281941041445
 0.19942873294459074
 0.1994284461131272